Copyright 2021 The Balloon Learning Environment Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# BLE View generated wind fields

This notebook allows you to sample from our trained WindField VAE and generate an animation of the resulting wind fields.

Simply run the cells in order!

In [ ]:
# @title Install necessary libraries
!pip install --upgrade pip
!pip install -U balloon_learning_environment

In [2]:
# @title Necessary imports
import datetime as dt
import numpy as np
import jax
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
rc('animation', html='jshtml')

from balloon_learning_environment.env import generative_wind_field

In [4]:
# @title Generate a wind field
wind_field = generative_wind_field.GenerativeWindField()
rng = jax.random.PRNGKey(0)
wind_field.reset_forecast(rng, dt.datetime.now())

In [ ]:
# @title Generate animation object (with a test still)
lats = np.linspace(-150, 150, 21)
lngs = np.linspace(-150, 150, 21)
pressure_levels = np.linspace(5000, 14000, 10)
x, y, z = np.meshgrid(lngs, lats, pressure_levels)
xx, yy = np.meshgrid(lats, lngs)

time_step = 0
direction = 1
episode_num = 0
elev = 10
azim = -60

fig = plt.figure(figsize=(20, 15))

def frame(w):
  global time_step
  global direction
  global episode_num
  global azim

  u = wind_field.field[:, :, :, time_step, 0]
  v = wind_field.field[:, :, :, time_step, 1]

  ax = fig.add_subplot(projection='3d', elev=elev, azim=azim)
  ax.quiver(x, y, z, u, v, np.zeros_like(u), length=1.5,
            arrow_length_ratio=0.01, normalize=False)
  ax.set_xlabel('Longitude displacement (km)')
  ax.set_ylabel('Latitude displacement (km)')
  ax.set_zlabel('Pressure (Pa)')
  ax.invert_zaxis()

  time_step += direction
  azim += 1
  if time_step == 9:
    time_step = 7
    direction = -1
    episode_num += 1
  elif time_step == -1:
    time_step = 1
    direction = 1
    episode_num += 1
    rng = jax.random.PRNGKey(episode_num)
    wind_field.reset_forecast(rng, dt.datetime.now())

  return ax
  
anim = animation.FuncAnimation(fig, frame, frames=40, blit=False, repeat=True)

Run one of the two cells below. To avoid large notebooks we are not embedding the generated videos, but you can find them in our [generated directory](https://github.com/google/balloon-learning-environment/tree/master/balloon_learning_environment/generated).

In [ ]:
# @title Display the animation in this notebook.
anim

In [ ]:
# @title Save the animation (you can access it from the left panel).
filename = 'wind_field.mp4'  # @param {type: 'string'}
anim.save(filename)